[목표]
- 500명의 키와 몸무게, 비만도 라벨을 이용하여 비만을 판단하는 모델을 만들어보자
- 머신러닝 전체 과정의 흐름을 이해해보자

### 문제정의
- 머신러닝을 통해서 어떤 문제를  해결할 것인가?
- 지도 vs 비지도 vs 강화
- 지도 : 정답과 문제를 알려주고 선택, 분류와 회귀로 구분 됨.
- 비만을 판단하는 모델을 만들자 ==> 지도학습 - 분류

### 데이터 수집
- csv 파일로 이루어진 데이터수집
- 500명의 성별, 키, 몸무게, 비만도 등급

In [4]:
# 데이터 꺼내오기
import pandas as pd
bmi=pd.read_csv('./Data/bmi_500.csv')
bmi

,Gender,Height,Weight,Label
0,Male,174,96,Obesity
1,Male,189,87,Normal
2,Female,185,110,Obesity
3,Female,195,104,Overweight
4,Male,149,61,Overweight
...,...,...,...,...
495,Female,150,153,Extreme Obesity
496,Female,184,121,Obesity
497,Female,141,136,Extreme Obesity
498,Male,150,95,Extreme Obesity


### 데이터 전처리
- 1. 결측치 처리 : 비어있는 데이터를 채워주기
- 2. 이상치 처리 : 정상적인 범위를 벗언나 데이터 수정하기

#### 결측치 확인
- 데이터 프레임의 형태를 갖춘후 info() 함수 사용

In [5]:
bmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Gender  500 non-null    object
 1   Height  500 non-null    int64 
 2   Weight  500 non-null    int64 
 3   Label   500 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


#### 이상치 확인
- 데이터 프레임의 형태를 갖춘 후 describe() 함수 사용
- describe() : 수치형 데이터만 분석가능, 기술통계

In [7]:
bmi.describe()
# count : 데이터 수
# mean : 평균 값
# std : 표준 편차
# min : 최소값
# 25% : 1분위수 
# 50% : 중앙값
# 75% : 3분위수
# max : 최대값

# 1, min, max 확인하기
# 2. 평균(mean), 중앙값(50%) 확인하기 ==> 두 수치가 비슷한지 확인
# 3. 두 수치가(평균, 중앙값)가 비슷하지 않을 경우 : min ~ max까지 5개의 간격을 확인하기 
# ==> 각 구간의 데이터가 고르게 분포가 되어있는지 확인 


# 우리의 데이틑 정상적으로 판단이 됨.

,Height,Weight
count,500.000000,500.000000
mean,169.944000,106.000000
std,16.375261,32.382607
min,140.000000,50.000000
25%,156.000000,80.000000
50%,170.500000,106.000000
75%,184.000000,136.000000
max,199.000000,160.000000


### 데이터 탐색 분석(EDA)

### 모델 선택및 하이퍼파라미터 튜닝

### 학습

### 평가